In [1]:
import zpu
import itertools
import math

In [2]:
def Ei(i):
    i = int(i)
    ei = [0,0,0]
    ei[i] = str(i)
    ei[(i+1)%3] = str((2+i)%3)
    ei[(i+2)%3] = str((1+i)%3)
    return ei
    

In [3]:
def isEipal(seq, i):
    "kontroluje jestli řetězec je pseudopalindrom"
    ei = Ei(i)
    l = len(seq)
    if l == 1:
        if seq == str(i):
            return True
        else:
            return False
    for x in range(0, math.ceil(l/2)):
        if seq[x] != ei[int(seq[l-1-x])]:
            return False
    return(True)

In [4]:
def makeEipalClosure (seq, i):
    "udělá z řetězce pseudopalindromický uzávěr"
    ei = Ei(i)
    if isEipal(seq, i) == True:
        return(seq)
    j = 1
    while isEipal(seq[j:], i) != True:
        j = j+1
    #print("    {0} nejdelší pseudopalindromický sufix : {1}".format(seq,seq[j:]))
    closure = seq
    pref = seq[j-1::-1]
    for letter in pref:
        closure = closure + ei[int(letter)]
    return(closure)

In [5]:
def make012Word(delta, theta, steps, seed = ""):
    "vytvoří slovo pomocí řídící posloupnosti a posloupnosti uzávěrů"
    w = seed
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = zpu.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        #print("w{0} = {1}".format(step+1,w))
    return(w)

In [8]:
def is012Normalized(delta, theta, steps):
    "kontroluje jestli muzou byt delts a theta normalizovane"
    w = ""
    l=1
    prefixes = []
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = zpu.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        prefixes.append(w)
    #print(prefixes)
    #print(w)
    while l <= len(w):
        prefix = w[:l]
        if zpu.isPal(prefix) or isEipal(prefix,0) or isEipal(prefix,0) or isEipal(prefix,0):
            if prefix in prefixes:
                #print(prefix + ": OK")
                pass
            else:
                #print("prefixy neobsahuji: " + prefix)
                return False
        l=l+1
    return True

In [10]:
is012Normalized("0100001","R012RR2", 7)

True

In [12]:
# Počet kroků
steps = 3
# Všechny kombinace pro delty a thety
thetas = [''.join(i) for i in itertools.product('R012', repeat=steps)]
deltas = [''.join(i) for i in itertools.product('012', repeat=steps)]
print(thetas)
print(deltas)

['RRR', 'RR0', 'RR1', 'RR2', 'R0R', 'R00', 'R01', 'R02', 'R1R', 'R10', 'R11', 'R12', 'R2R', 'R20', 'R21', 'R22', '0RR', '0R0', '0R1', '0R2', '00R', '000', '001', '002', '01R', '010', '011', '012', '02R', '020', '021', '022', '1RR', '1R0', '1R1', '1R2', '10R', '100', '101', '102', '11R', '110', '111', '112', '12R', '120', '121', '122', '2RR', '2R0', '2R1', '2R2', '20R', '200', '201', '202', '21R', '210', '211', '212', '22R', '220', '221', '222']
['000', '001', '002', '010', '011', '012', '020', '021', '022', '100', '101', '102', '110', '111', '112', '120', '121', '122', '200', '201', '202', '210', '211', '212', '220', '221', '222']


In [14]:
for theta in thetas:
    for delta in deltas:
        print("Theta: "+ theta + " Delta: " + delta +  " ===> " + make012Word(delta, theta, steps))
        print(is012Normalized(delta, theta, steps))
        print("")

Theta: RRR Delta: 000 ===> 000
True

Theta: RRR Delta: 001 ===> 00100
True

Theta: RRR Delta: 002 ===> 00200
True

Theta: RRR Delta: 010 ===> 010010
True

Theta: RRR Delta: 011 ===> 01010
True

Theta: RRR Delta: 012 ===> 0102010
False

Theta: RRR Delta: 020 ===> 020020
True

Theta: RRR Delta: 021 ===> 0201020
False

Theta: RRR Delta: 022 ===> 02020
True

Theta: RRR Delta: 100 ===> 10101
True

Theta: RRR Delta: 101 ===> 101101
True

Theta: RRR Delta: 102 ===> 1012101
True

Theta: RRR Delta: 110 ===> 11011
True

Theta: RRR Delta: 111 ===> 111
True

Theta: RRR Delta: 112 ===> 11211
True

Theta: RRR Delta: 120 ===> 1210121
False

Theta: RRR Delta: 121 ===> 121121
False

Theta: RRR Delta: 122 ===> 12121
False

Theta: RRR Delta: 200 ===> 20202
True

Theta: RRR Delta: 201 ===> 2021202
True

Theta: RRR Delta: 202 ===> 202202
True

Theta: RRR Delta: 210 ===> 2120212
False

Theta: RRR Delta: 211 ===> 21212
False

Theta: RRR Delta: 212 ===> 212212
False

Theta: RRR Delta: 220 ===> 22022
True

The

In [ ]:
isEipal("012", 1)

In [ ]:
a = [0,1, "ahoj"]
a = [a, "dalsi"]